# Sensor Footprint Generator

This notebook generates GeoJSON files representing the coverage areas (footprints) for each sensor based on satellite positions where data was received.

## What it does:
1. **Loads satellite data** from feather file
2. **Filters valid data points** (satellite altitude > 100km)
3. **Validates coordinates** (removes obviously false positions like 0,0)
4. **Creates coverage polygons** using convex hull algorithm
5. **Exports GeoJSON files** for each sensor

## Output:
- Individual GeoJSON files for each sensor in `sensor_geojson_cleaned/` directory
- Files can be used with mapping libraries and GIS software
- Each file contains coverage polygon and metadata

In [ ]:
import pandas as pd
import numpy as np
import pyarrow.feather as feather
import json
import ast
from scipy.spatial import ConvexHull
import os

In [ ]:
feather_path = "../data/parsed/ira.feather"
df = pd.DataFrame(feather.read_feather(feather_path))

In [ ]:
# Filter data for valid satellite positions (altitude > 100km)
print("Filtering data for valid satellite positions...")
valid_data = df[df['sat_alt'] > 100].copy()
print(f"Filtered data: {len(valid_data):,} rows (from {len(df):,} total)")
print(f"Valid data per sensor:")
sensor_counts = valid_data['sensor_name'].value_counts()
print(sensor_counts)

In [ ]:
def parse_coordinates_fixed(coord):
    """Parse coordinates - handle both numpy arrays and string representations"""
    try:
        if isinstance(coord, np.ndarray):
            return coord[0], coord[1]  # lat, lon
        elif isinstance(coord, (list, tuple)):
            return coord[0], coord[1]  # lat, lon
        elif isinstance(coord, str):
            coords = ast.literal_eval(coord)
            return coords[0], coords[1]  # lat, lon
        else:
            return None, None
    except:
        return None, None

def is_valid_coordinate_improved(lat, lon):
    """Enhanced validation to filter out obviously false positions"""
    # Check for exact zero coordinates (0.0, 0.0)
    if lat == 0.0 and lon == 0.0:
        return False
    
    # Check for reasonable latitude range (-90 to 90)
    if lat < -90 or lat > 90:
        return False
        
    # Check for reasonable longitude range (-180 to 180)
    if lon < -180 or lon > 180:
        return False
    
    # Filter out coordinates exactly on equator (suspicious for satellite positions)
    if lat == 0.0:
        return False
        
    # Filter out coordinates exactly on prime meridian (suspicious for satellite positions)
    if lon == 0.0:
        return False
    
    # Filter out coordinates very close to (0,0) - likely invalid
    if abs(lat) < 0.01 and abs(lon) < 0.01:
        return False
        
    return True

def create_geojson_polygon(points):
    """Create a GeoJSON polygon from a list of (lat, lon) points"""
    if len(points) < 3:
        return None
    
    # Convert to numpy array for ConvexHull
    coords_array = np.array(points)
    
    try:
        hull = ConvexHull(coords_array)
        # Get hull points in order
        hull_points = coords_array[hull.vertices]
        # Convert back to [lon, lat] format for GeoJSON (note the order!)
        geojson_coords = [[point[1], point[0]] for point in hull_points]
        # Close the polygon by adding the first point at the end
        geojson_coords.append(geojson_coords[0])
        
        return {
            "type": "Polygon",
            "coordinates": [geojson_coords]
        }
    except:
        # If convex hull fails, create a simple polygon from all points
        geojson_coords = [[point[1], point[0]] for point in coords_array]
        geojson_coords.append(geojson_coords[0])  # Close polygon
        return {
            "type": "Polygon", 
            "coordinates": [geojson_coords]
        }

In [ ]:
# Generate cleaned GeoJSON files for each sensor
print("Generating cleaned GeoJSON files for sensor footprints...")

# Create output directory for cleaned data
output_dir = "sensor_geojson_cleaned"
os.makedirs(output_dir, exist_ok=True)

for sensor_name in valid_data['sensor_name'].unique():
    print(f"\nProcessing sensor: {sensor_name}")
    
    # Get data for this sensor
    sensor_data = valid_data[valid_data['sensor_name'] == sensor_name]
    print(f"  Found {len(sensor_data):,} data points")
    
    # Extract and validate coordinates
    coordinates = []
    for _, row in sensor_data.iterrows():
        lat, lon = parse_coordinates_fixed(row['sat_pos_latlon'])
        if lat is not None and lon is not None:
            if is_valid_coordinate_improved(lat, lon):  # Use improved validation
                coordinates.append((lat, lon))
    
    original_count = len(sensor_data)
    filtered_count = len(coordinates)
    filtered_out = original_count - filtered_count
    
    print(f"  Valid coordinates: {filtered_count:,} (filtered out {filtered_out:,} invalid)")
    
    if len(coordinates) >= 3:  # Need at least 3 points for a polygon
        # Create GeoJSON polygon
        geometry = create_geojson_polygon(coordinates)
        
        if geometry:
            # Calculate statistics
            lats = [coord[0] for coord in coordinates]
            lons = [coord[1] for coord in coordinates]
            stats = {
                "data_points": len(coordinates),
                "original_data_points": original_count,
                "filtered_out": filtered_out,
                "filter_percentage": (filtered_out / original_count * 100) if original_count > 0 else 0,
                "lat_range": [min(lats), max(lats)],
                "lon_range": [min(lons), max(lons)],
                "center_lat": sum(lats) / len(lats),
                "center_lon": sum(lons) / len(lons)
            }
            
            # Create full GeoJSON feature
            geojson = {
                "type": "FeatureCollection",
                "features": [
                    {
                        "type": "Feature",
                        "properties": {
                            "sensor_name": sensor_name,
                            "data_points": len(coordinates),
                            "description": f"Coverage area for {sensor_name} based on satellite positions (sat_alt > 100km, invalid coordinates filtered)",
                            "statistics": stats,
                            "filtering_info": "Removed coordinates at (0,0), lat=0, lon=0, and other invalid positions"
                        },
                        "geometry": geometry
                    }
                ]
            }
            
            # Save to file
            filename = f"{output_dir}/{sensor_name.replace(' ', '_').replace('/', '_')}_coverage.geojson"
            with open(filename, 'w') as f:
                json.dump(geojson, f, indent=2)
            
            print(f"  ✓ Created GeoJSON file: {filename}")
            print(f"    Lat range: {stats['lat_range'][0]:.2f}° to {stats['lat_range'][1]:.2f}°")
            print(f"    Lon range: {stats['lon_range'][0]:.2f}° to {stats['lon_range'][1]:.2f}°")
        else:
            print(f"  ✗ Failed to create geometry for {sensor_name}")
    else:
        print(f"  ✗ Not enough coordinates ({len(coordinates)}) to create polygon for {sensor_name}")

print(f"\nCompleted! GeoJSON files saved in '{output_dir}/' directory")

In [ ]:
# Display summary and file information
print("=== SENSOR FOOTPRINT GENERATION COMPLETE ===")
print()

# List created files
print("Generated GeoJSON files:")
if os.path.exists(output_dir):
    geojson_files = [f for f in os.listdir(output_dir) if f.endswith('.geojson')]
    for file in sorted(geojson_files):
        file_path = os.path.join(output_dir, file)
        file_size = os.path.getsize(file_path) / 1024  # Size in KB
        print(f"  ✓ {file} ({file_size:.1f} KB)")
else:
    print("  No files found - run the previous cell first")

print(f"Coordinate filtering applied:")
print(f"  • Satellite altitude > 100km")
print(f"  • Removed coordinates at (0.0, 0.0)")
print(f"  • Removed coordinates with latitude = 0.0° (equator)")
print(f"  • Removed coordinates with longitude = 0.0° (prime meridian)")
print(f"  • Removed coordinates outside valid Earth bounds")

print(f"\nOutput directory: '{output_dir}/'")